In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets


In [2]:
# 데이터 불러오기
digits = datasets.load_digits()

digits

{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [3]:
X = digits.data

X

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [4]:
y = digits.target

y

array([0, 1, 2, ..., 8, 9, 8])

In [5]:
# 정규화
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# 넘파이 배열을 PyTorch 텐서로 변환 (데이터 타입을 float32로 지정)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# TensorDataset 객체로 입력 데이터와 레이블을 묶어줌 (데이터셋 생성)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# DataLoader 객체 생성: 데이터를 배치 크기만큼 묶고, 훈련 데이터는 섞어줌
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [8]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1437, 64), (360, 64), (1437,), (360,))

In [9]:
# 분류 모델 클래스 정의 (PyTorch의 nn.Module을 상속받음)
class ClassificationModel(nn.Module):
    def __init__(self):
        # 부모 클래스의 초기화 메서드 호출
        super(ClassificationModel, self).__init__()
        
        # 신경망 구조 정의: nn.Sequential을 사용해 레이어들을 순차적으로 연결
        self.model = nn.Sequential(
            nn.Linear(64, 64),   # 입력층: 입력 노드 64개, 은닉층 노드 64개
            nn.ReLU(),           # 활성화 함수: ReLU
            nn.Linear(64, 32),   # 은닉층: 64개 노드 -> 32개 노드
            nn.ReLU(),           # 활성화 함수: ReLU
            nn.Linear(32, 10)    # 출력층: 32개 노드 -> 10개 클래스 (0~9)
        )

    # 순전파 함수: 입력 데이터를 모델에 통과시켜 최종 출력값을 반환
    def forward(self, x):
        return self.model(x)

# 학습에 사용할 디바이스 설정 (GPU가 있으면 GPU 사용, 없으면 CPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델 인스턴스 생성 및 디바이스로 이동
model = ClassificationModel().to(device)

# 손실 함수 정의 (다중 분류이므로 CrossEntropyLoss 사용)
criterion = nn.CrossEntropyLoss()

# 옵티마이저 정의 (Adam 옵티마이저, 학습률: 0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
# 모델을 학습 모드로 전환 (드롭아웃, 배치 정규화 등의 레이어가 학습 모드로 동작)
model.train()

# 학습을 50번 반복 (에포크 수: 50)
for epoch in range(50):
    # 에포크마다 누적되는 손실 값을 초기화
    total_loss = 0
    
    # 학습 데이터셋에서 배치 단위로 데이터를 불러옴
    for X_batch, y_batch in train_loader:
        # 배치 데이터를 학습에 사용할 디바이스(CPU 또는 GPU)로 이동
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # 옵티마이저의 이전 그래디언트 초기화
        optimizer.zero_grad()
        
        # 모델에 입력 데이터를 전달하여 예측값(outputs) 계산
        outputs = model(X_batch)
        
        # 예측값과 실제값 간의 손실 계산
        loss = criterion(outputs, y_batch.squeeze())
        
        # 역전파를 통해 그래디언트 계산
        loss.backward()
        
        # 옵티마이저를 통해 모델 파라미터 업데이트
        optimizer.step()
        
        # 현재 배치의 손실을 누적
        total_loss += loss.item()
    
    # 에포크가 끝날 때마다 평균 손실을 출력
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")



Epoch 1, Loss: 2.1202
Epoch 2, Loss: 1.3046
Epoch 3, Loss: 0.5556
Epoch 4, Loss: 0.2736
Epoch 5, Loss: 0.1741
Epoch 6, Loss: 0.1277
Epoch 7, Loss: 0.0992
Epoch 8, Loss: 0.0794
Epoch 9, Loss: 0.0648
Epoch 10, Loss: 0.0537
Epoch 11, Loss: 0.0443
Epoch 12, Loss: 0.0371
Epoch 13, Loss: 0.0313
Epoch 14, Loss: 0.0267
Epoch 15, Loss: 0.0224
Epoch 16, Loss: 0.0197
Epoch 17, Loss: 0.0166
Epoch 18, Loss: 0.0142
Epoch 19, Loss: 0.0126
Epoch 20, Loss: 0.0110
Epoch 21, Loss: 0.0095
Epoch 22, Loss: 0.0086
Epoch 23, Loss: 0.0073
Epoch 24, Loss: 0.0064
Epoch 25, Loss: 0.0059
Epoch 26, Loss: 0.0053
Epoch 27, Loss: 0.0048
Epoch 28, Loss: 0.0043
Epoch 29, Loss: 0.0039
Epoch 30, Loss: 0.0036
Epoch 31, Loss: 0.0033
Epoch 32, Loss: 0.0031
Epoch 33, Loss: 0.0028
Epoch 34, Loss: 0.0026
Epoch 35, Loss: 0.0024
Epoch 36, Loss: 0.0022
Epoch 37, Loss: 0.0021
Epoch 38, Loss: 0.0019
Epoch 39, Loss: 0.0018
Epoch 40, Loss: 0.0017
Epoch 41, Loss: 0.0016
Epoch 42, Loss: 0.0015
Epoch 43, Loss: 0.0014
Epoch 44, Loss: 0.00

In [11]:
# 모델 평가 모드로 전환 (Dropout 등 비활성화)
model.eval()

# 예측값, 실제값, 확률값 저장 리스트 초기화
preds, actuals, probs_list = [], [], []

# 평가 시에는 그래디언트 계산 비활성화 (메모리 절약 및 연산 속도 향상)
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        # 배치 데이터를 디바이스에 전달
        X_batch = X_batch.to(device)
        
        # 모델에 입력 데이터를 통과시켜 출력값 계산
        outputs = model(X_batch)
        
        # 출력값에 소프트맥스 함수를 적용하여 각 클래스에 대한 확률값 계산
        probs = torch.softmax(outputs, dim=1).cpu().numpy()

        # 예측 레이블 계산 (가장 큰 확률값의 인덱스가 예측 레이블)
        pred_labels = np.argmax(probs, axis=1)
        preds.extend(pred_labels)  # 예측 레이블 리스트에 추가

        # 실제 레이블 저장 (CPU로 이동하여 numpy 형식으로 저장)
        actuals.extend(y_batch.cpu().numpy())

        # 확률값 저장
        probs_list.extend(probs)

# 리스트 형태의 확률값을 numpy 배열로 변환
probs_array = np.array(probs_list)

# 평가 지표 계산 (정확도, F1 스코어)
print("Accuracy:", accuracy_score(actuals, preds))
print("F1 Score:", f1_score(actuals, preds, average='weighted'))

# AUC 계산 (다중 클래스일 경우 `multi_class` 파라미터 설정 필요)
# `probs_array`의 형상은 `(샘플 수, 클래스 수)`이어야 함
print("AUC Score:", roc_auc_score(actuals, probs_array, multi_class='ovr'))


Accuracy: 0.975
F1 Score: 0.9750203567991709
AUC Score: 0.9992108990063014
